In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
import TipSlicer
import os
import scipy.ndimage as im
from importlib import reload
import tables

In [3]:
# load STL mesh and get X,Y, Z range
stlName='/home/sam/ULB/Notebooks/TipSlicer/StanfordDragonBinary.stl'
rootName='StanfordDragon'

lowresSlicing=1
lowresHashing=1

mesh,Zmins,Zmaxs=TipSlicer.loadStl(stlName,centred=True,mulDimension=100)
X,Y,Z=TipSlicer.getMeshXYZ(mesh,lowresHashing,lowresSlicing)
print('X: {} -> {}  Y: {} -> {}  Z: {} -> {}'.format(X[0].min(),X[0].max(),Y[:,0].min(),Y[:,0].max(),Z.min(),Z.max()))
print(X.shape,Y.shape,Z.shape)



X: -710 -> 710  Y: -320 -> 320  Z: 0.0 -> 972.0
(641, 1421) (641, 1421) (973,)


In [4]:
Zmins.max(),Zmaxs.max()

(969.13324, 970.85333)

In [14]:
# Do a first low resolution full scale stack with a correct infill
lowresH5=rootName+'_lowres.h5'
TipSlicer.stlToStack(mesh,lowresH5,X,Y,Z,Zmins,Zmaxs)

In [18]:
%%%time
overlapZ=2
overlapXY_multiplier=3 # Final overlap will be 2 times this multiplied with hashing
slicing=1
hashing=1
amShells=1
ScaffStep=2 # for solid infill => 2*hatching is spacing here
hatchStep=1 # for contour distance and top-bottom infill, not used if amShells==1 and doTopBottom=False


FOVrad=150  # more or less standard value but should be input
FOVz=300


lowresSlicing=1 # should try to get an automatic estimate for this based on largest workable matrix
lowresHashing=1

CodeSpeeds=np.ones(3)*30000     # should just use any value. Will be adjusted manually in the end
CodeIntensities=np.ones(3)*80

doGwl=True
doStack=False
doLowRes=False
doColourH5=False


if (not os.path.exists(rootName)):
    os.mkdir(rootName)
genBlockName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.h5'
genGwlName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.gwl'
genRegex=rootName+'/.+gwl'
h5Regex=rootName+'/.+\d\.h5' # should end in a digit, so as to avoid taking the _col.h5 files..
                             # alternatively, I could take them, and just reject upon inspection of the h5 file..
fullGwlName=rootName+'.gwl'
lowresH5=rootName+'_lowres.h5'


# load STL mesh and get X,Y, Z range
mesh,Zmins,Zmaxs=TipSlicer.loadStl(stlName,centred=True,mulDimension=100)
X,Y,Z=TipSlicer.getMeshXYZ(mesh,lowresHashing,lowresSlicing)
print('X: {} - {}  Y: {} - {}  Z: {} - {}'.format(X[0].min(),X[0].max(),Y[:,0].min(),Y[:,0].max(),Z.min(),Z.max()))
print(X.shape,Y.shape,Z.shape)

# Do a first low resolution full scale stack with a correct infill
if (doLowRes):
    TipSlicer.stlToStack(mesh,lowresH5,X,Y,Z,Zmins,Zmaxs)

if (doStack):
    # calculate/decide on Zrange for stitching (smallest integer number depending on FovZ and Zmax)
    amZ=int((Z.max()-Z.min())/FOVz)+1
    Zrange=(Z.max()-Z.min())/amZ
    print(amZ,Zrange)

    # loop over different Z-slices
    centreCoords=[]
    for k in range(amZ):
        Zstart=k*Zrange
        Zstop=(k+1)*Zrange+overlapZ
        zz=np.arange(Zstart,Zstop+slicing,slicing)
        sliceIndList= np.arange(len(zz))# adapt for variable slicing

        # now get low-resolution projection figures for this Z-range
        Ground,X,Y=TipSlicer.getProjection(lowresH5,Zstart,Zstop)

        # doOptimalStitching or hexagonal/square and find blockcentres
        optPoints=TipSlicer.getOptimalStitchCentres(Ground,FOVrad,FOVrad,X,Y)

        centreCoords.append(np.column_stack((optPoints,np.ones(len(optPoints))*Zstart)))


    # now try to order the points for optimal trajectory
    # do this before making the blocks so that you could implement a slant angle in the right direction..
    # start with TSP per layer, never mind the connection
    ordCentres=[]
    for optPoints in centreCoords:
        outPoint=np.zeros((len(optPoints),3),dtype=int)
        outPoint[:,0]=np.arange(len(optPoints)).astype(int)
        outPoint[:,1]=optPoints[:,0].astype(int)
        outPoint[:,2]=optPoints[:,1].astype(int)
        savetxt('myTest.txt',outPoint,fmt='%d')
        !./tsp myTest.txt
        TT=np.loadtxt('mysolution.txt')
        TT=TT[1:]
        ordOptPoints=outPoint[TT.astype(int),1:]
        ordCentres.append(ordOptPoints)



    # now creates the stacks
    for k in range(amZ):
        Zstart=k*Zrange
        Zstop=(k+1)*Zrange+overlapZ
        zz=np.arange(Zstart,Zstop+slicing,slicing)
        sliceIndList= np.arange(len(zz))# adapt for variable slicing


        optPoints=ordCentres[k][:,:2]
        # generate voronoi polygons
        regions, vertices = TipSlicer.makeVoronoiPolygons(optPoints)

        # do loop over blocks and write out the stacks (needs stl and low-res stack) 
        #                                        => CAN I USE formulaToStack and give stl-name and low-res stack as parameters??
        fullDist=[]
        zCentres=[]
        for i in np.arange(len(optPoints)):
    #    for i in np.arange(2):
            # now make coordinate systems
            print('Writing block {} in layer {}'.format(i,k))
            xc=optPoints[i,0]
            yc=optPoints[i,1]

            polygon = vertices[regions[i]]

            am=np.int(np.round(2*FOVrad/hashing))+1
            xx=np.linspace(xc-FOVrad,xc+FOVrad,am)
            yy=np.linspace(yc-FOVrad,yc+FOVrad,am)
            Xloc,Yloc=np.meshgrid(xx,yy)

            MM=TipSlicer.makeVoronoiMask(polygon,Xloc,Yloc)
            # if you need overlap, do binary_dilation on the mask
            MM=im.binary_dilation(MM,iterations=overlapXY_multiplier) # overlap = 2*iterations*hatching

            # with these coordinate systems, apply stlToStack
            blockName=genBlockName.format(k,i)

            TipSlicer.stlToStack(mesh,blockName,Xloc,Yloc,zz[sliceIndList],Zmins,Zmaxs,inFillMaster=lowresH5,writingMask=MM)
        
# and write the GWL's
# this part is same as formula-based stitching
if (doGwl):
    fp=os.path.split(h5Regex)
    if (fp[0]==''):
        fnames,fnumbs=TipSlicer.findFiles(h5Regex)
    else:
        fnames,fnumbs=TipSlicer.findFiles(fp[1],fp[0])
        
    for ff in fnames:
        # should decide on the sliceIndList here. It could differ from the stack, but this is difficult to determine here
        # especially if it needs to be in Z => which Z's are we doing here? Unknown in advance.
        gwlName=ff[:-3]+'.gwl'
        dists=TipSlicer.stackToGwl(ff,gwlName,amShells,ScaffStep,hatchStep
                                   ,writeHeader=False,atAngles=-1,writeColourH5=doColourH5,doTopBottom=False)
    if (len(dists)>0):
        if (len(fullDist)==0):
            fullDist=dists
        else:
            fullDist=fullDist+dists
    
    # Write out regex-gwl to follow this path.
    TipSlicer.writeGwlForRegex(fullGwlName,genRegex,CodeSpeeds,CodeIntensities)
    writeTime=(fullDist/CodeSpeeds[:len(fullDist)]).sum()/3600.
    print('Printing will take +- {:.2f} hours'.format(writeTime))






X: -710 - 710  Y: -320 - 320  Z: 0.0 - 972.0
(641, 1421) (641, 1421) (973,)
Printing will take +- 0.00 hours
CPU times: user 7min 34s, sys: 314 ms, total: 7min 34s
Wall time: 7min 34s


In [18]:
%%%time
overlapZ=2
overlapXY_multiplier=3 # Final overlap will be 2 times this multiplied with hashing
slicing=1
hashing=1
amShells=1
ScaffStep=2 # for solid infill => 2*hatching is spacing here
hatchStep=1 # for contour distance and top-bottom infill, not used if amShells==1 and doTopBottom=False


FOVrad=150  # more or less standard value but should be input
FOVz=300


lowresSlicing=1 # should try to get an automatic estimate for this based on largest workable matrix
lowresHashing=1

CodeSpeeds=np.ones(3)*30000     # should just use any value. Will be adjusted manually in the end
CodeIntensities=np.ones(3)*80

doGwl=True
doStack=False
doLowRes=False
doColourH5=False


if (not os.path.exists(rootName)):
    os.mkdir(rootName)
genBlockName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.h5'
genGwlName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.gwl'
genRegex=rootName+'/.+gwl'
h5Regex=rootName+'/.+\d\.h5' # should end in a digit, so as to avoid taking the _col.h5 files..
                             # alternatively, I could take them, and just reject upon inspection of the h5 file..
fullGwlName=rootName+'.gwl'
lowresH5=rootName+'_lowres.h5'


# load STL mesh and get X,Y, Z range
mesh,Zmins,Zmaxs=FormulaSlicer.loadStl(stlName,centred=True,mulDimension=100)
X,Y,Z=FormulaSlicer.getMeshXYZ(mesh,lowresHashing,lowresSlicing)
print('X: {} - {}  Y: {} - {}  Z: {} - {}'.format(X[0].min(),X[0].max(),Y[:,0].min(),Y[:,0].max(),Z.min(),Z.max()))
print(X.shape,Y.shape,Z.shape)

# Do a first low resolution full scale stack with a correct infill
if (doLowRes):
    FormulaSlicer.stlToStack(mesh,lowresH5,X,Y,Z,Zmins,Zmaxs)

if (doStack):
    # calculate/decide on Zrange for stitching (smallest integer number depending on FovZ and Zmax)
    amZ=int((Z.max()-Z.min())/FOVz)+1
    Zrange=(Z.max()-Z.min())/amZ
    print(amZ,Zrange)

    # loop over different Z-slices
    centreCoords=[]
    for k in range(amZ):
        Zstart=k*Zrange
        Zstop=(k+1)*Zrange+overlapZ
        zz=np.arange(Zstart,Zstop+slicing,slicing)
        sliceIndList= np.arange(len(zz))# adapt for variable slicing

        # now get low-resolution projection figures for this Z-range
        Ground,X,Y=FormulaSlicer.getProjection(lowresH5,Zstart,Zstop)

        # doOptimalStitching or hexagonal/square and find blockcentres
        optPoints=FormulaSlicer.getOptimalStitchCentres(Ground,FOVrad,FOVrad,X,Y)

        centreCoords.append(np.column_stack((optPoints,np.ones(len(optPoints))*Zstart)))


    # now try to order the points for optimal trajectory
    # do this before making the blocks so that you could implement a slant angle in the right direction..
    # start with TSP per layer, never mind the connection
    ordCentres=[]
    for optPoints in centreCoords:
        FormulaSlicer.writeTSPfile('blocks.tsp',optPoints[:,:2])
        !concorde -o 'tsp.sol' 'blocks.tsp'
        Ord=FormulaSlicer.readTSPfile('tsp.sol')
        ordCentres.append(optPoints[Ord])
        
        
    # now creates the stacks
    for k in range(amZ):
        Zstart=k*Zrange
        Zstop=(k+1)*Zrange+overlapZ
        zz=np.arange(Zstart,Zstop+slicing,slicing)
        sliceIndList= np.arange(len(zz))# adapt for variable slicing


        optPoints=ordCentres[k][:,:2]
        # generate voronoi polygons
        regions, vertices = FormulaSlicer.makeVoronoiPolygons(optPoints)

        # do loop over blocks and write out the stacks (needs stl and low-res stack) 
        #                                        => CAN I USE formulaToStack and give stl-name and low-res stack as parameters??
        fullDist=[]
        zCentres=[]
        for i in np.arange(len(optPoints)):
    #    for i in np.arange(2):
            # now make coordinate systems
            print('Writing block {} in layer {}'.format(i,k))
            xc=optPoints[i,0]
            yc=optPoints[i,1]

            polygon = vertices[regions[i]]

            am=np.int(np.round(2*FOVrad/hashing))+1
            xx=np.linspace(xc-FOVrad,xc+FOVrad,am)
            yy=np.linspace(yc-FOVrad,yc+FOVrad,am)
            Xloc,Yloc=np.meshgrid(xx,yy)

            MM=FormulaSlicer.makeVoronoiMask(polygon,Xloc,Yloc)
            # if you need overlap, do binary_dilation on the mask
            MM=im.binary_dilation(MM,iterations=overlapXY_multiplier) # overlap = 2*iterations*hatching

            # with these coordinate systems, apply stlToStack
            blockName=genBlockName.format(k,i)

            FormulaSlicer.stlToStack(mesh,blockName,Xloc,Yloc,zz[sliceIndList],Zmins,Zmaxs,inFillMaster=lowresH5,writingMask=MM)
        
# and write the GWL's
# this part is same as formula-based stitching
if (doGwl):
    fp=os.path.split(h5Regex)
    if (fp[0]==''):
        fnames,fnumbs=FormulaSlicer.findFiles(h5Regex)
    else:
        fnames,fnumbs=FormulaSlicer.findFiles(fp[1],fp[0])
        
    for ff in fnames:
        # should decide on the sliceIndList here. It could differ from the stack, but this is difficult to determine here
        # especially if it needs to be in Z => which Z's are we doing here? Unknown in advance.
        gwlName=ff[:-3]+'.gwl'
        dists=FormulaSlicer.stackToGwl(ff,gwlName,amShells,ScaffStep,hatchStep
                                   ,writeHeader=False,atAngles=-1,writeColourH5=doColourH5,doTopBottom=False)
    if (len(dists)>0):
        if (len(fullDist)==0):
            fullDist=dists
        else:
            fullDist=fullDist+dists
    
    # Write out regex-gwl to follow this path.
    FormulaSlicer.writeGwlForRegex(fullGwlName,genRegex,CodeSpeeds,CodeIntensities)
    writeTime=(fullDist/CodeSpeeds[:len(fullDist)]).sum()/3600.
    print('Printing will take +- {:.2f} hours'.format(writeTime))






X: -710 - 710  Y: -320 - 320  Z: 0.0 - 972.0
(641, 1421) (641, 1421) (973,)
Printing will take +- 0.00 hours
CPU times: user 7min 34s, sys: 314 ms, total: 7min 34s
Wall time: 7min 34s


In [ ]:
overlapZ=2
overlapXY_multiplier=3 # Final overlap will be 2 times this multiplied with hashing
slicing=1
hashing=1
amShells=1
ScaffStep=2 # for solid infill => 2*hatching is spacing here
hatchStep=1 # for contour distance and top-bottom infill, not used if amShells==1 and doTopBottom=False


FOVrad=150  # more or less standard value but should be input
FOVz=300


lowresSlicing=1 # should try to get an automatic estimate for this based on largest workable matrix
lowresHashing=1

CodeSpeeds=np.ones(3)*30000     # should just use any value. Will be adjusted manually in the end
CodeIntensities=np.ones(3)*80

doGwl=True
doStack=False
doLowRes=False
doColourH5=False


if (not os.path.exists(rootName)):
    os.mkdir(rootName)
genBlockName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.h5'
genGwlName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.gwl'
genRegex=rootName+'/.+gwl'
h5Regex=rootName+'/.+\d\.h5' # should end in a digit, so as to avoid taking the _col.h5 files..
                             # alternatively, I could take them, and just reject upon inspection of the h5 file..
fullGwlName=rootName+'.gwl'
lowresH5=rootName+'_lowres.h5'


# load STL mesh and get X,Y, Z range
mesh,Zmins,Zmaxs=FormulaSlicer.loadStl(stlName,centred=True,mulDimension=100)
X,Y,Z=FormulaSlicer.getMeshXYZ(mesh,lowresHashing,lowresSlicing)
print('X: {} - {}  Y: {} - {}  Z: {} - {}'.format(X[0].min(),X[0].max(),Y[:,0].min(),Y[:,0].max(),Z.min(),Z.max()))
print(X.shape,Y.shape,Z.shape)

In [5]:
amZ=int((Z.max()-Z.min())/FOVz)+1
Zrange=(Z.max()-Z.min())/amZ
print(amZ,Zrange)

# loop over different Z-slices
centreCoords=[]
for k in range(amZ):
    Zstart=k*Zrange
    Zstop=(k+1)*Zrange+overlapZ
    zz=np.arange(Zstart,Zstop+slicing,slicing)
    sliceIndList= np.arange(len(zz))# adapt for variable slicing

    # now get low-resolution projection figures for this Z-range
    Ground,X,Y=FormulaSlicer.getProjection(lowresH5,Zstart,Zstop)

    # doOptimalStitching or hexagonal/square and find blockcentres
    optPoints=FormulaSlicer.getOptimalStitchCentres(Ground,FOVrad,FOVrad,X,Y)

    centreCoords.append(np.column_stack((optPoints,np.ones(len(optPoints))*Zstart)))


NameError: name 'FOVz' is not defined

In [19]:
fullDist

array([ 258.66963058,  303.99998047])